In [100]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader

In [102]:
# ----------------------------
# 1️⃣ 데이터 로드 및 전처리 (UCI HAR Raw Data)
# ----------------------------

def load_ucihar_data(base_path="E:/dataset/HAR/UCI-HAR"):
    def load_signals(folder, dataset):
        signals = []
        signal_types = [
            "body_acc_x", "body_acc_y", "body_acc_z",
            "body_gyro_x", "body_gyro_y", "body_gyro_z",
            "total_acc_x", "total_acc_y", "total_acc_z"
        ]
        for signal in signal_types:
            filename = f"{base_path}/{dataset}/Inertial Signals/{signal}_{dataset}.txt"
            data = np.loadtxt(filename).reshape(-1, 128, 1)  # (samples, time_steps, channels)
            signals.append(data)
        return np.concatenate(signals, axis=2)  # (samples, 128, 9)

    # Raw 센서 데이터 로드 (9개의 센서)
    X_train_raw = load_signals("train", "train")  # (train_samples, 128, 9)
    X_test_raw = load_signals("test", "test")  # (test_samples, 128, 9)

    # Feature 데이터 (561개)
    X_train_features = np.loadtxt(f"{base_path}/train/X_train.txt")
    X_test_features = np.loadtxt(f"{base_path}/test/X_test.txt")

    # 최종 입력 데이터 결합 (CNN & LSTM에 맞게 변형)
    X_train = np.expand_dims(X_train_raw, axis=1)  # (samples, 1, 128, 9) → CNN 입력 형태
    X_test = np.expand_dims(X_test_raw, axis=1)

    # 레이블 데이터 로드
    y_train = np.loadtxt(f"{base_path}/train/y_train.txt").astype(int) - 1
    y_test = np.loadtxt(f"{base_path}/test/y_test.txt").astype(int) - 1

    # Tensor 변환
    X_train = torch.tensor(X_train, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.long)
    y_test = torch.tensor(y_test, dtype=torch.long)

    return X_train, y_train, X_test, y_test

# 데이터 로드
X_train, y_train, X_test, y_test = load_ucihar_data()
output_dim = 6  # 6개 클래스
epochs = 50
learning_rate = 0.001

In [110]:
class Expert(nn.Module):
    def __init__(self, num_channels=9, hidden_dim=64, output_dim=6, num_heads=4):
        super(Expert, self).__init__()

        # CNN Layer (Feature Extraction)
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(5, 1), stride=1, padding=(2, 0))
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3, 1), stride=1, padding=(1, 0))
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d((2, 1))  # (batch, 64, 64, 9)

        # Transformer Encoder (Global Context)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=64, nhead=num_heads, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)

        # Fully Connected Layer
        self.fc = nn.Linear(64, output_dim)

    def forward(self, x):
        # CNN Feature Extraction
        x = self.relu(self.conv1(x))  # (batch, 32, 128, 9)
        x = self.relu(self.conv2(x))  # (batch, 64, 128, 9)
        x = self.pool(x)  # (batch, 64, 64, 9)

        # CNN 출력 차원 조정 → Transformer 입력 형태 (batch, time_steps=64, features=64)
        x = x.permute(0, 2, 3, 1).contiguous()  # (batch, 64, 9, 64) → (batch, 64, 64, 9)
        x = x.mean(dim=2)  # (batch, 64, 64), 센서 채널 평균

        # Transformer Encoder
        x = self.transformer_encoder(x)  # (batch, 64, 64)

        # Mean Pooling over Time-axis
        x = torch.mean(x, dim=1)  # (batch, 64)

        # Fully Connected Layer
        return torch.softmax(self.fc(x), dim=1)


In [112]:
# ----------------------------
# 3️⃣ Gating Network (6개 Expert 선택)
# ----------------------------

class Gating(nn.Module):
    def __init__(self, input_dim, num_experts):
        super(Gating, self).__init__()
        self.layer1 = nn.Linear(input_dim, 128)
        self.layer2 = nn.Linear(128, 256)
        self.layer3 = nn.Linear(256, 128)
        self.layer4 = nn.Linear(128, num_experts)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = torch.relu(self.layer3(x))
        return torch.softmax(self.layer4(x), dim=1)  # Expert 가중치

In [114]:
# ----------------------------
# 4️⃣ Mixture of Experts 모델
# ----------------------------

class MoE(nn.Module):
    def __init__(self, experts):
        super(MoE, self).__init__()
        self.experts = nn.ModuleList(experts)
        self.gating = Gating(input_dim=experts[0].fc.in_features, num_experts=len(experts))

    def forward(self, x):
        weights = self.gating(x.mean(dim=1))  # Gating Network가 Expert 가중치 결정
        expert_outputs = torch.stack([expert(x) for expert in self.experts], dim=2)  # 모든 Expert의 출력값을 stack
        weights = weights.unsqueeze(1).expand_as(expert_outputs)  # 가중치 형태 맞추기
        return torch.sum(expert_outputs * weights, dim=2)  # Expert 출력 조합하여 최종 예측

In [ ]:
# ----------------------------
# 5️⃣ Expert 학습
# ----------------------------

experts = [Expert() for _ in range(6)]
optimizers = [optim.Adam(expert.parameters(), lr=learning_rate) for expert in experts]

for i, expert in enumerate(experts):
    optimizer = optimizers[i]
    mask = y_train == i
    x_train_expert, y_train_expert = X_train[mask], y_train[mask]  

    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = expert(x_train_expert)
        loss = nn.CrossEntropyLoss()(outputs, y_train_expert)
        loss.backward()
        optimizer.step()


In [ ]:
# ----------------------------
# 6️⃣ MoE 학습
# ----------------------------

moe_model = MoE(experts)
optimizer_moe = optim.Adam(moe_model.parameters(), lr=learning_rate)

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)

for epoch in range(epochs):
    for batch_x, batch_y in train_loader:
        optimizer_moe.zero_grad()
        outputs_moe = moe_model(batch_x)
        loss_moe = nn.CrossEntropyLoss()(outputs_moe, batch_y)
        loss_moe.backward()
        optimizer_moe.step()

In [ ]:
# ----------------------------
# 7️⃣ 모델 평가
# ----------------------------

def evaluate(model, X, y):
    with torch.no_grad():
        outputs = model(X)
        _, predicted = torch.max(outputs, 1)
        accuracy = (predicted == y).sum().item() / len(y)
    return accuracy

accuracy_experts = [evaluate(expert, X_test, y_test) for expert in experts]
accuracy_moe = evaluate(moe_model, X_test, y_test)

for i, acc in enumerate(accuracy_experts):
    print(f"Expert {i+1} Accuracy: {acc:.4f}")
print(f"Mixture of Experts Accuracy: {accuracy_moe:.4f}")